# IMPORTS AND DATA

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install openai requests

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import numpy as np
import pandas as pd
import requests
from sklearn.metrics import accuracy_score
import transformers
import torch
from openai import OpenAI
from huggingface_hub import notebook_login
notebook_login()

from sklearn.metrics.pairwise import cosine_similarity
from torchtext.vocab import GloVe

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_data = pd.read_pickle('drive/MyDrive/Colab Notebooks/cgfp_cleaned_embeddings_data_test_500.pkl')

,unnamed: 0,product type,food product group,food product category,primary food product category,product name,basic type,sub-type 1,sub-type 2,flavor/cut,...,cooked/cleaned,wg/wgr,dietary concern,additives,dietary accommodation,frozen,packaging,commodity,basic type corrected,product type embeddings
8572,14799,Bread Flat Whole Grain Garlic K-12 Commodity C...,"Bread, Grains & Legumes",Grain Products,Grain Products,"flatbread, garlic, whole grain rich",flatbread,garlic,NaN,NaN,...,NaN,whole grain rich,NaN,NaN,NaN,NaN,NaN,NaN,flatbread,"[-1.924228973686695, 6.084960013628006, 0.1091..."
32116,57741,Ground Sage,Condiments & Snacks,Condiments & Snacks,Condiments & Snacks,"spice, sage",spice,sage,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spice,"[-1.395110011100769, 0.9157600104808807, -0.91..."
54413,94068,MILK SKIM,Milk & Dairy,Milk,Milk,"milk, nonfat",milk,NaN,NaN,NaN,...,NaN,NaN,nonfat,NaN,NaN,NaN,NaN,NaN,milk,"[-0.01302000880241394, 0.9080000221729279, -0...."
52612,90710,Cheese Parm Shaved Usa,Milk & Dairy,Cheese,Cheese,"cheese, parmesan, shaved",cheese,parmesan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheese,"[0.4846799820661545, 0.6984899826347828, -0.07..."
61930,108549,"ORANGES, CARACARA WEDGE 50x2oz",Produce,Fruit,Fruit,"orange, cara cara, wedge",orange,cara cara,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,orange,"[-0.7006719931960106, -0.1729240007698536, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69434,122765,BROCCOLI--FLORETTES 3#--BG,Produce,Vegetables,Vegetables,"broccoli, floret",broccoli,floret,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,broccoli,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
60428,105296,059774 LEMON SPLT 12 CT,Produce,Fruit,Fruit,lemon,lemon,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lemon,"[-0.9739339388906956, 0.9118799865245819, -0.9..."
65416,115027,CARROT PEELED SLIM 4/5#,Produce,Roots & Tubers,Roots & Tubers,"carrot, peeled",carrot,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,carrot,"[-2.695240080356598, 0.6365490257740021, 0.322..."
33358,60097,KS CHOCOLATE CHIPS 4.5LBS SEMl-5WEETP270,Condiments & Snacks,Condiments & Snacks,Condiments & Snacks,"topping, chocolate chip",topping,chocolate chip,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,topping,"[-0.6699800044298172, 1.8106699585914612, -1.0..."


In [ ]:
training_df = pd.read_pickle('drive/MyDrive/Colab Notebooks/cgfp_cleaned_embeddings_data_train.pkl')

,unnamed: 0,product type,food product group,food product category,primary food product category,product name,basic type,sub-type 1,sub-type 2,flavor/cut,...,cooked/cleaned,wg/wgr,dietary concern,additives,dietary accommodation,frozen,packaging,commodity,basic type corrected,product type embeddings
28977,51296,"MIX, SAUCE DEMI-GLACE ADD WATER CAN SHELF STABLE",Condiments & Snacks,Condiments & Snacks,Condiments & Snacks,"sauce, steak",sauce,steak,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sauce,"[-4.684709995985031, 3.4759150138124824, 1.307..."
65101,114401,"CARROTS STICKS 3/8""x3/8""x4"" 5 LB",Produce,Roots & Tubers,Roots & Tubers,"carrot, stick",carrot,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,carrot,"[-1.3133400082588196, 2.703510046005249, -0.37..."
2343,3869,JUICE ORANGE CUP 41380,Beverages,Beverages,Beverages,"juice, orange, ss",juice,NaN,NaN,flavored,...,NaN,NaN,NaN,NaN,NaN,NaN,ss,NaN,juice,"[-1.2619899809360504, 1.2153199911117554, -0.1..."
72535,129225,BASIL OPAL 4 OZ,Produce,Vegetables,Vegetables,"herb, basil, opal",herb,basil,opal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,herb,"[0.10734003782272339, 1.812662061303854, -1.38..."
37231,66521,"NOODLE, YKSOB WHL WHEAT CKD",Meals,Meals,Meals,"meal, noodle, yakisoba, whole wheat",meal,noodle,yakisoba,NaN,...,NaN,whole grain rich,NaN,NaN,NaN,NaN,NaN,NaN,meal,"[-0.7355799973011017, 0.8441699743270874, 1.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6464,11244,"BREADSTICK, FRENCH UNSLICED BAKED FROZEN","Bread, Grains & Legumes",Grain Products,Grain Products,"breadstick, frozen",breadstick,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,frozen,NaN,NaN,breadstick,"[-1.249107986688614, 0.9079170078039169, 0.110..."
56900,98594,APPLE FUJI 100 CT,Produce,Fruit,Fruit,"apple, fuji",apple,fuji,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,apple,"[-0.6701560281217098, -0.07919996976852417, -0..."
79260,144368,VEGETABLE BLEND CALIF GOLD CHEF CUT,Produce,Vegetables,Vegetables,"vegetable, blend",vegetable,NaN,NaN,blend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vegetable,"[-0.6322699300944805, 3.1004899740219116, 0.32..."
875,1336,Pepsi Max Syrup BIB 1/3 Ga,Beverages,Beverages,Beverages,"drink, soda, cola, mix",drink,soda,cola,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drink,"[-0.9638300016522408, 2.1478789672255516, 0.06..."


In [ ]:
#stored in secrets, have not deleted yet because I want to make sure I did it right

# Get Relevant Examples

In [ ]:
# Loading the GloVe Embeddings
glove_embeddings = GloVe(name='6B', dim=100)

# Function to get the GloVe embedding for a string
def get_embedding(string, glove_embeddings):
  """Returns glove embedding for string
  string: any character str
  returns: vector of embeddings"""
  words = string.lower().split()
  embedding = np.zeros(glove_embeddings.dim)
  for word in words:
      if word in glove_embeddings.stoi:
          embedding += glove_embeddings[word].numpy()
  return embedding

# Function to calculate cosine similarity
def calculate_cosine_similarity(embedding1, embedding2):
  """Returns cosine similarity between two vectors
  embedding1: vector
  embedding2: vector
  returns: cosine similarity"""
  if np.linalg.norm(embedding1) == 0 or np.linalg.norm(embedding2) == 0:
      return 0
  return cosine_similarity([embedding1], [embedding2])[0][0]

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:43<00:00, 9134.61it/s]


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

# Normalizing the embeddings
embeddings = np.array(training_df['product type embeddings'].tolist())
normalized_embeddings = normalize(embeddings)

# Fitting Nearest Neighbors using BallTree
nbrs = NearestNeighbors(algorithm='ball_tree', metric='euclidean').fit(normalized_embeddings)

# Function to find the product with the closest cosine similarity using Nearest Neighbors
def get_similar_products(new_product_type, df, glove_embeddings, nbrs, n=1):
    """
    Find the top n products with the closest cosine similarity to a given product type using Nearest Neighbors.

    Input:
        new_product_type (str): The product type to find similar products for.
        df (pd.DataFrame): The DataFrame containing product data and embeddings.
        glove_embeddings (GloVe): The GloVe embeddings model.
        nbrs (NearestNeighbors): The fitted Nearest Neighbors model.
        n (int, optional): The number of top similar products to return. Defaults to 1.

    Returns:
        pd.DataFrame: A DataFrame containing the top n similar products and their cosine similarity scores.
    """
    # Generating the embedding for the new product type
    new_product_embedding = get_embedding(new_product_type, glove_embeddings)
    new_product_embedding_normalized = normalize(new_product_embedding.reshape(1, -1))

    # Finding the top n nearest neighbors
    distances, indices = nbrs.kneighbors(new_product_embedding_normalized, n_neighbors=n)

    # Getting the top matches from the DataFrame
    top_matches = df.iloc[indices[0]].copy()  #takes the distances between the points that we calculated
    top_matches.loc[:, 'Cosine Similarity'] = 1 - distances[0]  # Convert distances to cosine similarity (since we normalized), sort the closest nearest neighbors by cosine similairty

    return top_matches

In [ ]:
get_similar_products('chocolate cake', training_df, glove_embeddings, nbrs, n=3)

,unnamed: 0,product type,food product group,food product category,primary food product category,product name,basic type,sub-type 1,sub-type 2,flavor/cut,...,wg/wgr,dietary concern,additives,dietary accommodation,frozen,packaging,commodity,basic type corrected,product type embeddings,Cosine Similarity
24677,43784,CHOCOLATE CREAM CAKE MIX,Condiments & Snacks,Condiments & Snacks,Condiments & Snacks,"mix, cake, chocolate",mix,cake,NaN,flavored,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mix,"[-2.5141900181770325, 2.322070002555847, -1.25...",0.691905
55026,95148,85001-RICHS CHOCOLATE,Milk & Dairy,Milk & Dairy,Milk & Dairy,"ice cream, bar, chocolate",ice cream,bar,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ice cream,"[-0.147599995136261, 0.8278399705886841, -0.49...",0.681253
3981,7033,"SUPPLEMENT, MIGHTY-SHAKES CHOCOLATE",Beverages,Beverages,Beverages,"supplement, chocolate",supplement,NaN,NaN,flavored,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,supplement,"[-0.147599995136261, 0.8278399705886841, -0.49...",0.681253


In [ ]:
def convert_to_example_list(top_matches):
  """Converts simillar products to list of examples
  top_matches: dataframe with top n similar products
  returns: list of tuples with product type and basic type"""
  example_list = []
  for index, row in top_matches.iterrows():
      a_tuple = (row['product type'], row['basic type'])
      example_list.append(a_tuple)
  return example_list

In [ ]:
print(convert_to_example_list(get_similar_products('chocolate cake', training_df, glove_embeddings, nbrs, n=3)))

[('CHOCOLATE CREAM CAKE MIX', 'mix'), ('85001-RICHS CHOCOLATE', 'ice cream'), ('SUPPLEMENT, MIGHTY-SHAKES CHOCOLATE', 'supplement')]


# MAKE PROMPT FUNCTION

In [ ]:
# Function to create a prompt based on product type
def make_prompt(product_type, example_list):
  """This function turns product type into prompt.
  product_type: string with product type
  example_list: list of relevant examples,(product_type, basic_type)
  return: string with prompt
  """

  # Initialize an empty list to store the formatted examples
  formatted_examples = []

  # Iterate through the list of examples, "example" contains the tuples
  for example in example_list:
    example_product_type, example_basic_type = example  #extract tuple values, example is the product_type and its corresponding basic type
    formatted_examples.append(f"- For the product name '{example_product_type}', the basic type is '{example_basic_type}'.\n")

  # Join all formatted examples into a single string
  examples_str = ''.join(formatted_examples)

  prompt = (   # takes in the product_type from each column in the dataset, {product_type} is a placeholder, it will be replaced by the actual product type
        f"Given the product name '{product_type}', classify it into a basic type. The basic type is the general category of the product, such as water, juice. \n"

        #specifiying desired length
        "Do not give a reasoning for your answer. Just return the resulting basic type.\n"


        "For example:\n"
        f"{examples_str}"  # Insert formatted examples here

        "Use your knowledge of the specific product to classify them into a basic type to the best of your ability.\n"
        "Remember, only return the resulting basic type in lower case and nothing else. Only provide the word that is associated with the basic type. Classify desserts as 'dessert', and meats as their own unique classifications (beef, pork, turkey, etc.). Be very specific, do not give any general basic type results. Use the words in the product type to help guide you in determining the basic type; oftentimes the desired basic type is a word within the product type/name.\n"
    )

  return prompt  # returns the updated prompt that includes the specified product type within it




In [ ]:
print(make_prompt('chocolate cake', convert_to_example_list(get_similar_products('chocolate cake', training_df, glove_embeddings, nbrs, n=3))))

Given the product name 'chocolate cake', classify it into a basic type. The basic type is the general category of the product, such as water, juice. 
Do not give a reasoning for your answer. Just return the resulting basic type.
For example:
- For the product name 'CHOCOLATE CREAM CAKE MIX', the basic type is 'mix'.
- For the product name '85001-RICHS CHOCOLATE', the basic type is 'ice cream'.
- For the product name 'SUPPLEMENT, MIGHTY-SHAKES CHOCOLATE', the basic type is 'supplement'.
Use your knowledge of the specific product to classify them into a basic type to the best of your ability.
Remember, only return the resulting basic type in lower case and nothing else. Only provide the word that is associated with the basic type. Classify desserts as 'dessert', and meats as their own unique classifications (beef, pork, turkey, etc.). Be very specific, do not give any general basic type results. Use the words in the product type to help guide you in determining the basic type; oftentimes

# QUERY LLAMA FUNCTION



In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
def query_llama(prompt):
  """This function queries the Llama model
  prompt: string with query
  return: string with response from model
  """
    # Create a list of messages that simulate a conversation.
    # The "system" message sets the overall instruction context, and the "user" message includes the prompt.
  messages = [
        {"role": "system", "con tent": "Answer questions in a single phrase or singular category name.\n"
        "If unsure, provide the closest category."},
        {"role": "user", "content": prompt}
    ]
    # Apply the chat template to the messages.
    # This step tokenizes the messages and adds any necessary formatting for the model.
  prompt_object = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # Generate a response using the language model.
    # The model generates up to 256 new tokens with some randomness (temperature=0.6, top_p=0.9).
  outputs = pipeline(
        prompt_object,
        max_new_tokens=256,
        eos_token_id=pipeline.tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    # Extract the generated text from the output.
  generated_text = outputs[0]["generated_text"]

    # Remove the prompt from the generated text to get the final response.
  response = generated_text[len(prompt_object):].strip().lower()
  return response

# QUERY OPENAI FUNCTION

In [ ]:
# Function to query OpenAI GPT model
def query_openai(prompt):
  """This function queries the OpenAI GPT model
  prompt: string with query
  return: string with response from model
  """
  client = OpenAI(api_key=openai_api_key)
  response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Answer questions in a single phrase or singular category name.\n"
            "If unsure, provide the closest category."},
            {"role": "user", "content": prompt
            }]
    )
  return response.choices[0].message.content.strip().lower()


# GENERAL PURPOSE FUNCTION THAT QUERIES EITHER MODEL

In [ ]:
# GENERAL PURPOSE FUNCTION
def query_llm(prompt, model):
  """This function queries the OpenAI GPT model or the Llama model
  prompt: string with query
  model: string with model name, either "openai" or "llama"
  return: string with response from model
  """
  if model == "llama":
      result = query_llama(prompt)  # Call query_llama with the prompt to get the basic type from Llama model
      return result
  elif model == "openai":
      result = query_openai(prompt) # Call query_openai with the prompt to get the basic type from OpenAI model
      return result
  else:
      response = "Unknown model"

      return response

# GET PREDICTIONS w/ RAG

In [ ]:
test_data['Relevant Examples'] = test_data.apply(lambda row: convert_to_example_list(get_similar_products(row['product type'], training_df, glove_embeddings, nbrs, n=3)), axis=1)

In [ ]:
test_data['Prompt'] = test_data.apply(lambda row: make_prompt(row['product type'], row['Relevant Examples']), axis=1)

In [ ]:
test_data.iloc[0, -1]

'Given the product name \'Bread Flat Whole Grain Garlic K-12 Commodity Capable\', classify it into a basic type. The basic type is the general category of the product, such as water, juice. \nDo not give a reasoning for your answer. Just return the resulting basic type.\nFor example:\n- For the product name \'Breadsticks Cheese Whole Grain Reduced Fat (C) K-12 Commodity Capable\', the basic type is \'breadstick\'.\n- For the product name \'Pizza Cheese 4" Round Whole Grain 4.46Oz K-12 Commodity Capable\', the basic type is \'pizza\'.\n- For the product name \'Pizza Cheese Big Daddy\'S Harvest 51% Whole Grain K-12 Commodity Capable\', the basic type is \'pizza\'.\nUse your knowledge of the specific product to classify them into a basic type to the best of your ability.\nRemember, only return the resulting basic type in lower case and nothing else. Only provide the word that is associated with the basic type. Classify desserts as \'dessert\', and meats as their own unique classifications

In [ ]:
test_data['llama basic type'] = test_data.apply(lambda row: query_llm(row['Prompt'], 'llama'), axis=1)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_to

In [ ]:
test_data['openai basic type'] = test_data.apply(lambda row: query_llm(row['Prompt'], 'openai'), axis=1)

In [ ]:
test_data.head()

,unnamed: 0,product type,food product group,food product category,primary food product category,product name,basic type,sub-type 1,sub-type 2,flavor/cut,...,dietary accommodation,frozen,packaging,commodity,basic type corrected,product type embeddings,Relevant Examples,Prompt,llama basic type,openai basic type
8572,14799,Bread Flat Whole Grain Garlic K-12 Commodity C...,"Bread, Grains & Legumes",Grain Products,Grain Products,"flatbread, garlic, whole grain rich",flatbread,garlic,NaN,NaN,...,NaN,NaN,NaN,NaN,flatbread,"[-1.924228973686695, 6.084960013628006, 0.1091...",[(Breadsticks Cheese Whole Grain Reduced Fat (...,Given the product name 'Bread Flat Whole Grain...,bread,bread
32116,57741,Ground Sage,Condiments & Snacks,Condiments & Snacks,Condiments & Snacks,"spice, sage",spice,sage,NaN,NaN,...,NaN,NaN,NaN,NaN,spice,"[-1.395110011100769, 0.9157600104808807, -0.91...","[(Sage Ground 12ozSJ, herb), (SAGE LEAVES GROU...","Given the product name 'Ground Sage', classify...",herb,spice
54413,94068,MILK SKIM,Milk & Dairy,Milk,Milk,"milk, nonfat",milk,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,milk,"[-0.01302000880241394, 0.9080000221729279, -0....","[(Skim Milk, milk), (Milk Skim, milk), (Skim m...","Given the product name 'MILK SKIM', classify i...",milk,milk
52612,90710,Cheese Parm Shaved Usa,Milk & Dairy,Cheese,Cheese,"cheese, parmesan, shaved",cheese,parmesan,NaN,NaN,...,NaN,NaN,NaN,NaN,cheese,"[0.4846799820661545, 0.6984899826347828, -0.07...","[(CHEESE PARM SHAVED USA, cheese), (CHEESE PAR...",Given the product name 'Cheese Parm Shaved Usa...,cheese,cheese
61930,108549,"ORANGES, CARACARA WEDGE 50x2oz",Produce,Fruit,Fruit,"orange, cara cara, wedge",orange,cara cara,NaN,NaN,...,NaN,NaN,NaN,NaN,orange,"[-0.7006719931960106, -0.1729240007698536, -0....","[(FRIES, Wedge 10cut w/skn FRZ, french fry), (...","Given the product name 'ORANGES, CARACARA WEDG...",oranges,orange


# GET PREDICTIONS w/o RAG

In [ ]:
examples = [('CIDER SPRKLG MARTINELLI', 'apple cider'), ('YOGURT FRESH NON FAT LIGHT N FIT STRAWBERRY BLUE RASPBERRY', 'yogurt'), ('COFFEE, H&R BLND 32Z', 'coffee'), ('JUICE, APPL 100% VITMN C ADDED', 'juice'), ('TOMATO GRAPE RED 12X1 PT', 'tomato')]

In [ ]:
test_wo_rag = test_data[['product type', 'basic type']].copy()
test_wo_rag['Prompt'] = test_wo_rag.apply(lambda row: make_prompt(row['product type'], examples), axis=1)

In [ ]:
test_wo_rag.iloc[0, -1]

"Given the product name 'Bread Flat Whole Grain Garlic K-12 Commodity Capable', classify it into a basic type. The basic type is the general category of the product, such as water, juice. \nDo not give a reasoning for your answer. Just return the resulting basic type.\nFor example:\n- For the product name 'CIDER SPRKLG MARTINELLI', the basic type is 'apple cider'.\n- For the product name 'YOGURT FRESH NON FAT LIGHT N FIT STRAWBERRY BLUE RASPBERRY', the basic type is 'yogurt'.\n- For the product name 'COFFEE, H&R BLND 32Z', the basic type is 'coffee'.\n- For the product name 'JUICE, APPL 100% VITMN C ADDED', the basic type is 'juice'.\n- For the product name 'TOMATO GRAPE RED 12X1 PT', the basic type is 'tomato'.\nUse your knowledge of the specific product to classify them into a basic type to the best of your ability.\nRemember, only return the resulting basic type in lower case and nothing else. Only provide the word that is associated with the basic type. Classify desserts as 'desser

In [ ]:
test_wo_rag['llama basic type'] = test_wo_rag.apply(lambda row: query_llm(row['Prompt'], 'llama'), axis=1)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [ ]:
test_wo_rag['openai basic type'] = test_wo_rag.apply(lambda row: query_llm(row['Prompt'], 'openai'), axis=1)

In [ ]:
test_wo_rag.head()

,product type,basic type,Prompt,llama basic type,openai basic type
8572,Bread Flat Whole Grain Garlic K-12 Commodity C...,flatbread,Given the product name 'Bread Flat Whole Grain...,bread,bread
32116,Ground Sage,spice,"Given the product name 'Ground Sage', classify...",herb,spice
54413,MILK SKIM,milk,"Given the product name 'MILK SKIM', classify i...",milk,milk
52612,Cheese Parm Shaved Usa,cheese,Given the product name 'Cheese Parm Shaved Usa...,cheese,cheese
61930,"ORANGES, CARACARA WEDGE 50x2oz",orange,"Given the product name 'ORANGES, CARACARA WEDG...",oranges,oranges


# COMPARE PERFORMANCE w/ RAG

In [ ]:
result_df = test_data[['product type','basic type', 'openai basic type', 'llama basic type']]
result_df.head()

,product type,basic type,openai basic type,llama basic type
8572,Bread Flat Whole Grain Garlic K-12 Commodity C...,flatbread,bread,bread
32116,Ground Sage,spice,spice,herb
54413,MILK SKIM,milk,milk,milk
52612,Cheese Parm Shaved Usa,cheese,cheese,cheese
61930,"ORANGES, CARACARA WEDGE 50x2oz",orange,orange,oranges


In [ ]:
#LEMMENTIZE BEFORE GET EMBEDDINGS

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

  # Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the GloVe embedding for a string with lemmatization
def get_embedding(string, glove_embeddings):
    """Returns GloVe embedding for string after lemmatization
    string: any character str
    returns: vector of embeddings"""
    words = nltk.word_tokenize(string.lower())
    embedding = np.zeros(glove_embeddings.dim)
    for word in words:
        lemmatized_word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
        if lemmatized_word in glove_embeddings.stoi:
            embedding += glove_embeddings[lemmatized_word].numpy()
    return embedding



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Applying the get_embedding function to each column
result_df['Basic Type Embedding'] = result_df['basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
result_df['Open AI Basic Type Embedding'] = result_df['openai basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
result_df['LLAMA 3 Basic Type Embedding'] = result_df['llama basic type'].apply(lambda x: get_embedding(x, glove_embeddings))

<ipython-input-38-89facca78a46>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Basic Type Embedding'] = result_df['basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
<ipython-input-38-89facca78a46>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Open AI Basic Type Embedding'] = result_df['openai basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
<ipython-input-38-89facca78a46>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [ ]:
result_df['OpenAI Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['Open AI Basic Type Embedding']), axis=1)
result_df['Llama 3 Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['LLAMA 3 Basic Type Embedding']), axis=1)

<ipython-input-39-c2675776cbd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['OpenAI Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['Open AI Basic Type Embedding']), axis=1)
<ipython-input-39-c2675776cbd5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Llama 3 Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['LLAMA 3 Basic Type Embedding']), axis=1)


In [ ]:
def check_match(similarity, threshold):
  """Check if similarity is above threshold
  similarity: float with cosine similarity
  threshold: float with threshold
  return: 1 if similarity is above threshold, 0 otherwise
  """
  if similarity >= threshold:
    return 1

  else:
    return 0

In [ ]:
result_df['OpenAI Match'] = result_df.apply(lambda row: check_match(row['OpenAI Cosine Similarity'], .8), axis=1)

result_df['Llama 3 Match'] = result_df.apply(lambda row: check_match(row['Llama 3 Cosine Similarity'], .8), axis=1)

<ipython-input-41-85107b3f81dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['OpenAI Match'] = result_df.apply(lambda row: check_match(row['OpenAI Cosine Similarity'], .8), axis=1)
<ipython-input-41-85107b3f81dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Llama 3 Match'] = result_df.apply(lambda row: check_match(row['Llama 3 Cosine Similarity'], .8), axis=1)


In [ ]:
openai_accuracy = result_df['OpenAI Match'].sum() / len(result_df)
llama_accuracy = result_df['Llama 3 Match'].sum() / len(result_df)

In [ ]:
print("OpenAI Accuracy:", openai_accuracy)
print("Llama Accuracy:", llama_accuracy)

OpenAI Accuracy: 0.868
Llama Accuracy: 0.782


# COMPARE PERFORMANCE w/o RAG

In [ ]:
result_df = test_wo_rag[['product type','basic type', 'openai basic type', 'llama basic type']]
result_df.head()

,product type,basic type,openai basic type,llama basic type
8572,Bread Flat Whole Grain Garlic K-12 Commodity C...,flatbread,bread,bread
32116,Ground Sage,spice,spice,herb
54413,MILK SKIM,milk,milk,milk
52612,Cheese Parm Shaved Usa,cheese,cheese,cheese
61930,"ORANGES, CARACARA WEDGE 50x2oz",orange,oranges,oranges


In [ ]:
# Applying the get_embedding function to each column
result_df['Basic Type Embedding'] = result_df['basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
result_df['Open AI Basic Type Embedding'] = result_df['openai basic type'].apply(lambda x: get_embedding(x, glove_embeddings))
result_df['LLAMA 3 Basic Type Embedding'] = result_df['llama basic type'].apply(lambda x: get_embedding(x, glove_embeddings))

In [ ]:
result_df['OpenAI Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['Open AI Basic Type Embedding']), axis=1)
result_df['Llama 3 Cosine Similarity'] = result_df.apply(lambda row: calculate_cosine_similarity(row['Basic Type Embedding'], row['LLAMA 3 Basic Type Embedding']), axis=1)

In [ ]:
result_df['OpenAI Match'] = result_df.apply(lambda row: check_match(row['OpenAI Cosine Similarity'], .8), axis=1)

result_df['Llama 3 Match'] = result_df.apply(lambda row: check_match(row['Llama 3 Cosine Similarity'], .8), axis=1)

In [ ]:
openai_accuracy = result_df['OpenAI Match'].sum() / len(result_df)
llama_accuracy = result_df['Llama 3 Match'].sum() / len(result_df)

In [ ]:
print("OpenAI Accuracy:", openai_accuracy)
print("Llama Accuracy:", llama_accuracy)

OpenAI Accuracy: 0.702
Llama Accuracy: 0.584
